In [1]:
from IPython.display import display, HTML
import scipy.constants as sc
from scipy.optimize import fsolve
from ipywidgets import interact, widgets
%matplotlib notebook

import numpy as np
import cmath
import matplotlib.pyplot as plt
import scipy.constants as sc
plt.rcParams.update({
    "text.usetex": True,
    'text.latex.preamble': r"\usepackage{amsmath} \boldmath"
})
import os
import h5py

print(os.getpid())
%cd ../

12564
/media/work/docs/codes/QuCF/scripts-py


In [2]:
# --- Launch to change cells' width ---
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pylib.mix as mix
import pylib.qucf_read as qucf_r
import pylib.qucf_oracle as qucf_o
import pylib.qucf_matrix_D as qucf_m
import kin1d1d as kin
path_ = "../simulations/KIN1D1D/"
file_name_oracle_ = "circuit_OH"
path_test_ = "../simulations/test-simplified/"
path_cl_ = "../results/KIN1D1D-results/"
path_save_ = "../results/KIN1D1D-results/figs/"
file_name_oracle_ = "circuit_OH"

In [4]:
# ---------------------------------------------------------------
# --- Read the matrix ---
# ---------------------------------------------------------------
mix.reload_module(qucf_r)
mix.reload_module(kin)


print()
dd_44 = qucf_r.read_matrix(path_, "flat_44_OUTPUT.hdf5") #  nv = 4, nx = 4 

print()
dd_45 = qucf_r.read_matrix(path_, "flat_45_OUTPUT.hdf5") #  nv = 4, nx = 5 

print()
dd_54 = qucf_r.read_matrix(path_, "flat_54_OUTPUT.hdf5") #  nx = 5, nv = 4

print()
dd_55 = qucf_r.read_matrix(path_, "flat_55_OUTPUT.hdf5") #  nx = 5, nv = 5




print()
dd_64 = qucf_r.read_matrix(path_, "flat_64_OUTPUT.hdf5") #  nx = 6, nv = 4

# print()
# dd_65 = qucf_r.read_matrix(path_, "flat_65_OUTPUT.hdf5") #  nx = 6, nv = 5

# print()
# dd_66 = qucf_r.read_matrix(path_, "flat_66_OUTPUT.hdf5") #  nx = 6, nv = 6


Reading the matrix from: flat_44_OUTPUT.hdf5
from the path: ../simulations/KIN1D1D/
date of the simulation:  06-06-2023 08:57:47
matrix name:  U_BE
N = 512

Reading the matrix from: flat_45_OUTPUT.hdf5
from the path: ../simulations/KIN1D1D/
date of the simulation:  06-23-2023 15:42:09
matrix name:  U_BE
N = 1024

Reading the matrix from: flat_54_OUTPUT.hdf5
from the path: ../simulations/KIN1D1D/
date of the simulation:  06-23-2023 15:48:52
matrix name:  U_BE
N = 1024

Reading the matrix from: flat_55_OUTPUT.hdf5
from the path: ../simulations/KIN1D1D/
date of the simulation:  06-23-2023 15:52:48
matrix name:  U_BE
N = 2048

Reading the matrix from: flat_64_OUTPUT.hdf5
from the path: ../simulations/KIN1D1D/
date of the simulation:  07-31-2023 16:30:41
matrix name:  U_BE
N = 2048


In [9]:
# ---------------------------------------------------------------
# --- Initialize matrices D_F for various sizes ---
# ---------------------------------------------------------------
mix.reload_module(kin)
mix.reload_module(qucf_m)

grid_44 = qucf_m.SectionsGrid__(kin.init_matrix_and_circuit(dd_44))
grid_45 = qucf_m.SectionsGrid__(kin.init_matrix_and_circuit(dd_45))
grid_54 = qucf_m.SectionsGrid__(kin.init_matrix_and_circuit(dd_54))

In [10]:
# ---------------------------------------------------------------
# --- Create an extrapolation template ---
# ---------------------------------------------------------------
mix.reload_module(kin)
mix.reload_module(qucf_m)

oo_extr = qucf_m.Extrapolation__([grid_44, grid_45, grid_54])
oo_extr.create_extrapolation_template()

All grids have the same number of sizes.
The correct number of grids is provided.
Grids have similiar structure.


In [14]:
# ---------------------------------------------------------------
# --- Construct a large matrix ---
# ---------------------------------------------------------------
mix.reload_module(kin)
mix.reload_module(qucf_m)
mix.reload_module(qucf_o)

def compare_matrix(A_init, nx, nv):
    Nx = 1 << nx
    Nv = 1 << nv
    N = Nx * Nv
    A_final = A_init[0:N,0:N]
    return A_final

nx_large, nv_large = 6, 6

A_to_compare = compare_matrix(dd_66["A"], nx_large, nv_large)

circ_large = kin.init_circuit_of_defined_size(nx_large, nv_large, 3, 3)
D_large = oo_extr.reconstruct_matrix(circ_large)
qucf_o.compare_matrices(D_large, A_to_compare)

the same within precision  1e-06


In [18]:
# ---------------------------------------------------------------
# --- Function to print matrix subblocks ---
# ---------------------------------------------------------------
mix.reload_module(mix)
def print_submatrix(nx, nv, B):
    Nx = 1 << nx
    Nv = 1 << nv
    
    print("****** Nx, Nv: {:d}, {:d} *************************************".format(Nx, Nv))
    
    ir_x, ic_x = 0, 0 # x[0], diag
#     ir_x, ic_x = 0, 1 # V-f: x[0], diag+1
#     ir_x, ic_x = 0, 2 # V-f: x[0], diag+2
#     ir_x, ic_x = 0, 3 # V-f: x[0], diag+3

#     ir_x, ic_x = 1, 0 # V-f: x[1], diag-1
#     ir_x, ic_x = 1, 1 # V-f: x[1], diag
#     ir_x, ic_x = 1, 2 # V-f: x[1], diag+1
#     ir_x, ic_x = 1, 3 # V-f: x[1], diag+2

#     ir_x, ic_x = 2, 1 # V-f: x[1], diag-1
#     ir_x, ic_x = 2, 2 # V-f: x[1], diag
#     ir_x, ic_x = 2, 3 # V-f: x[1], diag+1

#     ir_x, ic_x = 3, 2 # V-f: x[1], diag-1
#     ir_x, ic_x = 3, 3 # V-f: x[1], diag
#     ir_x, ic_x = 3, 4 # V-f: x[1], diag+1

#     ir_x, ic_x = Nx-1, Nx-4 # V-f: x[1], diag-3
#     ir_x, ic_x = Nx-1, Nx-3 # V-f: x[1], diag-2
#     ir_x, ic_x = Nx-1, Nx-2 # V-f: x[1], diag-1
#     ir_x, ic_x = Nx-1, Nx-1 # V-f: x[1], diag

    # --------------------------------------------------
    Nr_print, Nc_print = Nv, Nv
    ir_start, ic_start = Nv * ir_x, Nv * ic_x
    
    mix.print_subblock_colored(
        B, 
        ir_start, ic_start,
        Nr_print, Nc_print,
        ff=[6, 3, "f"], 
        n_in_row = Nc_print, 
        gap_be = " ", 
    )
    
# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------
print_submatrix(nx_large, nv_large, D_large.real)  
print_submatrix(nx_large, nv_large, A_to_compare.real)

****** Nx, Nv: 64, 16 *************************************
--- 0:   0.062  0.044  0.044  0.031  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 1:   0.044  0.088  0.031  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 2:   0.000  0.044  0.062  0.031  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 3:   0.000  0.000  0.031  0.062  0.044  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 4:   0.000  0.000  0.000  0.031  0.088  0.044  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 5:   0.000  0.000  0.000  0.000  0.044  0.088  0.044  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 6:   0.000  0.000  0.000  0.000  0.000  0.044  0.088  0.044  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 7:   0.000  0.000  0.000  0.000  0.000  0.000  0.044  0.088  0.044  0.000  0.000  0.000  0.000  